# TEAM 5 - Software Design 1 | Model Training
This notebook is to demonstrate the models used in our software design about water quality classification.

## Data Preprocessing Pipeline

### Import libraries

In [ ]:
!pip install catboost

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,r2_score, mean_absolute_error, mean_squared_error, r2_score
import warnings
import os
import datetime
import time

### Import dataset
[Dataset](https://figshare.com/articles/dataset/A_Comprehensive_Surface_Water_Quality_Monitoring_Dataset_1940-2023_2_82Million_Record_Resource_for_Empirical_and_ML-Based_Research/27800394/2?file=50757303)

In [28]:
path = '/content/drive/MyDrive/SCHOOLFILES/CPE31/CPE025A/Chapters 2 and 3/code/27800394/Dataset/Combined Data/Combined_dataset.csv'
df = pd.read_csv(path)
df.tail(10) #show latest data

,Country,Area,Waterbody Type,Date,Ammonia (mg/l),Biochemical Oxygen Demand (mg/l),Dissolved Oxygen (mg/l),Orthophosphate (mg/l),pH (ph units),Temperature (cel),Nitrogen (mg/l),Nitrate (mg/l),CCME_Values,CCME_WQI
2827967,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,22-06-2021,0.042,1.0,11.684103,0.025,8.0,14.10,0.56,0.55,100.0,Excellent
2827968,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,07-10-2021,0.020,1.3,4.068000,0.019,7.6,13.50,0.51,0.51,100.0,Excellent
2827969,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,30-11-2021,0.047,1.0,4.972500,0.017,7.9,11.15,0.56,0.55,100.0,Excellent
2827970,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,08-02-2022,0.039,1.1,11.684103,0.013,7.9,8.30,0.77,0.76,100.0,Excellent
2827971,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,07-04-2022,0.030,1.0,11.684103,0.025,8.1,6.00,0.57,0.57,100.0,Excellent
2827972,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,21-06-2022,0.024,1.4,4.293000,0.025,7.9,16.20,0.37,0.37,100.0,Excellent
2827973,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,28-09-2022,0.038,1.4,4.495500,0.025,7.9,11.40,0.54,0.54,100.0,Excellent
2827974,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,29-11-2022,0.035,1.4,4.464000,0.016,7.6,6.20,0.80,0.79,100.0,Excellent
2827975,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,07-02-2023,0.046,1.4,4.176000,0.012,8.0,7.80,0.95,1.30,100.0,Excellent
2827976,Ireland,"Cork Harbour, Moy Killala, YELLOW (KNOCK)_020",River,11-04-2023,0.020,1.2,4.324500,0.025,7.9,7.90,0.49,1.30,100.0,Excellent


### Reading Dataset Dimensions and Feature Data Types

In [29]:
print("Number of datapoints (rows):", df.shape[0])
print("Number of columns:", df.shape[1])
print("\nData types:\n", df.dtypes)

Number of datapoints (rows): 2827977
Number of columns: 14

Data types:
 Country                              object
Area                                 object
Waterbody Type                       object
Date                                 object
Ammonia (mg/l)                      float64
Biochemical Oxygen Demand (mg/l)    float64
Dissolved Oxygen (mg/l)             float64
Orthophosphate (mg/l)               float64
pH (ph units)                       float64
Temperature (cel)                   float64
Nitrogen (mg/l)                     float64
Nitrate (mg/l)                      float64
CCME_Values                         float64
CCME_WQI                             object
dtype: object


### Check for empty or missing values


In [30]:
print("\nMissing values per column:\n", df.isnull().sum())


Missing values per column:
 Country                             0
Area                                0
Waterbody Type                      0
Date                                0
Ammonia (mg/l)                      0
Biochemical Oxygen Demand (mg/l)    0
Dissolved Oxygen (mg/l)             0
Orthophosphate (mg/l)               0
pH (ph units)                       0
Temperature (cel)                   0
Nitrogen (mg/l)                     0
Nitrate (mg/l)                      0
CCME_Values                         0
CCME_WQI                            0
dtype: int64


### Removal of unnecessary features/columns

In [31]:
print(list(df.columns))

['Country', 'Area', 'Waterbody Type', 'Date', 'Ammonia (mg/l)', 'Biochemical Oxygen Demand (mg/l)', 'Dissolved Oxygen (mg/l)', 'Orthophosphate (mg/l)', 'pH (ph units)', 'Temperature (cel)', 'Nitrogen (mg/l)', 'Nitrate (mg/l)', 'CCME_Values', 'CCME_WQI']


*The only columns needed are data pertaining to pH level, Nitrate concentration, and Ammonia concentration, as these are the components used in DENR water quality guidelines and standards that can also be tested by the client in remote areas through reagents for the aforementioned components.*

*So, that leaves us with 4 columns (the 3 columns for the components and the water classification)*

In [32]:
df_filtered = df[['Ammonia (mg/l)',
                  'pH (ph units)',
                  'Nitrate (mg/l)',
                  'CCME_WQI'
                ]]

df_filtered.tail(10)

,Ammonia (mg/l),pH (ph units),Nitrate (mg/l),CCME_WQI
2827967,0.042,8.0,0.55,Excellent
2827968,0.020,7.6,0.51,Excellent
2827969,0.047,7.9,0.55,Excellent
2827970,0.039,7.9,0.76,Excellent
2827971,0.030,8.1,0.57,Excellent
2827972,0.024,7.9,0.37,Excellent
2827973,0.038,7.9,0.54,Excellent
2827974,0.035,7.6,0.79,Excellent
2827975,0.046,8.0,1.30,Excellent
2827976,0.020,7.9,1.30,Excellent


### Prepare for modelling

In [36]:
X = df_filtered.drop('CCME_WQI', axis = 1)
y = df_filtered['CCME_WQI']

#Split into 2 sets: Training (70%) and Testing (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Training of Machine Learning Models

### RandomForest

In [38]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

# Initialize Random Forest with multiple cores and 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Measure training time
start_time = time.time()
rf.fit(X_train, y_train)
end_time = time.time()

# Predictions and probabilities on test set
predictions = rf.predict(X_test)
probs = rf.predict_proba(X_test)[:, 1]

# Evaluation metrics
print("Classification Report:\n", classification_report(y_test, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))
print("Random Forest Accuracy:", accuracy_score(y_test, predictions))
print("Random Forest ROC-AUC:", roc_auc_score(y_test, probs))
print(f"Random Forest training time: {end_time - start_time:.2f} seconds")


KeyboardInterrupt: 